# 2. Visualize PiP Participation Surfaces

This notebook visualizes subject-level node participation profiles generated by percolation-based network attack simulations (PiP).

For each subject:

- We load their `node_participation_at_percolation` matrix
- Crop to the longest contiguous block of valid values
- Apply weighting to emphasize early percolation steps
- Plot and save:
  - **2D Participation Curve Plot**: weighted curves for all nodes
  - **3D Participation Surface Plot**: node × step × participation


In [ ]:
import os
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Set data directory relative to repo
data_dir = os.path.join("..", "results")
fig_dir = os.path.join("..", "figures", "PiP_surfaces")
os.makedirs(fig_dir, exist_ok=True)

# List all participation files
output_files = sorted([f for f in os.listdir(data_dir) if f.endswith('_participation.mat')])


In [ ]:
for fname in output_files:
    fpath = os.path.join(data_dir, fname)
    print(f"Processing: {fname}")

    # Load matrix
    data = scipy.io.loadmat(fpath)
    if 'node_participation_at_percolation' not in data:
        print("Missing variable.")
        continue
    x = data['node_participation_at_percolation']

    # --- Crop to longest valid block ---
    valid_rows = ~np.isnan(np.sum(x, axis=1))
    diff = np.diff(np.concatenate([[0], valid_rows.astype(int), [0]]))
    starts = np.where(diff == 1)[0]
    ends = np.where(diff == -1)[0]
    if len(starts) == 0 or len(ends) == 0:
        print("No valid block.")
        continue
    longest = np.argmax(ends - starts)
    x_crop = x[starts[longest]:ends[longest], :]

    # --- Fill NaNs and weight ---
    x_crop = np.nan_to_num(x_crop, nan=0.0)
    flatten = -np.linspace(1 / len(x_crop), 1, len(x_crop))[:, None]
    weighted = (x_crop + flatten) * np.linspace(1, 1 / len(x_crop), len(x_crop))[:, None]
    weighted = np.clip(weighted, 0, None)

    # --- Plot and save 2D ---
    plt.figure(figsize=(10, 6))
    for i in range(weighted.shape[1]):
        plt.plot(weighted[:, i], linewidth=2, alpha=0.3)
    ax = plt.gca()
    ax.set_facecolor('white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(False)
    plt.xlabel('Percolation Step')
    plt.ylabel('Weighted Participation')
    plt.title(f'2D Participation Curve: {fname}')
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, fname.replace('.mat', '_2D.png')))
    plt.show()

    # --- Plot and save 3D ---
    steps, nodes = weighted.shape
    X, Y = np.meshgrid(np.arange(steps), np.arange(nodes))
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, weighted.T, cmap='RdYlBu_r', linewidth=0, antialiased=False)
    ax.set_title(f'3D Participation Surface: {fname}')
    ax.set_xlabel('Attack Step')
    ax.set_ylabel('Node Index')
    ax.set_zlabel('Participation')
    ax.set_xlim(0, steps - 1)
    ax.set_ylim(0, nodes - 1)
    ax.view_init(elev=30, azim=135)
    plt.tight_layout()
    plt.savefig(os.path.join(fig_dir, fname.replace('.mat', '_3D.png')))
    plt.show()


## Notes

- Participation matrices are saved under `results/` as `*_participation.mat`.
- Node participation values are visualized in both curve and surface formats.
- Output figures are saved to `figures/PiP_surfaces/` as `.png` files.
